In [1]:
import requests
import pprint
import pandas as pd

In [2]:
url1 = "https://nubanner.neu.edu/StudentRegistrationSsb/ssb/classSearch/getTerms?offset=1&max=14"

sems_json = requests.get(url1).json()

spring_sem_code = "202530"
sems_json

[{'code': '202610', 'description': 'Fall 2025 Semester (View Only)'},
 {'code': '202560', 'description': 'Summer 2 2025 Semester'},
 {'code': '202554', 'description': 'Summer 2025 CPS Semester'},
 {'code': '202550', 'description': 'Summer Full 2025 Semester'},
 {'code': '202540', 'description': 'Summer 1 2025 Semester'},
 {'code': '202535', 'description': 'Spring 2025 CPS Quarter'},
 {'code': '202534', 'description': 'Spring 2025 CPS Semester'},
 {'code': '202532', 'description': 'Spring 2025 Law Semester'},
 {'code': '202530', 'description': 'Spring 2025 Semester'},
 {'code': '202525', 'description': 'Winter 2025 CPS Quarter'},
 {'code': '202515', 'description': 'Fall 2024 CPS Quarter (View Only)'},
 {'code': '202514', 'description': 'Fall 2024 CPS Semester (View Only)'},
 {'code': '202512', 'description': 'Fall 2024 Law Semester (View Only)'},
 {'code': '202510', 'description': 'Fall 2024 Semester (View Only)'}]

In [3]:
url2 = f"https://nubanner.neu.edu/StudentRegistrationSsb/ssb/classSearch/get_subject?&term={spring_sem_code}&offset=1&max=300"

subjects_json = requests.get(url2).json()
subjects_json

[{'code': 'ACCT', 'description': 'Accounting'},
 {'code': 'AFCS', 'description': 'Africana Studies'},
 {'code': 'AMSL', 'description': 'American Sign Language'},
 {'code': 'ANTH', 'description': 'Anthropology'},
 {'code': 'ARAB', 'description': 'Arabic'},
 {'code': 'ARCH', 'description': 'Architecture'},
 {'code': 'ARMY', 'description': 'Army ROTC'},
 {'code': 'ARTG', 'description': 'Art - Design'},
 {'code': 'ARTF', 'description': 'Art - Fundamentals'},
 {'code': 'ARTE', 'description': 'Art - General'},
 {'code': 'ARTH', 'description': 'Art - History'},
 {'code': 'ARTD', 'description': 'Art - Media Arts'},
 {'code': 'ARTS', 'description': 'Art - Studio'},
 {'code': 'AACE', 'description': 'Arts Admin &amp; Cultural Entrep'},
 {'code': 'ASNS', 'description': 'Asian Studies'},
 {'code': 'BNSC', 'description': 'Behavioral Neuroscience'},
 {'code': 'BIOC', 'description': 'Biochemistry'},
 {'code': 'BIOE', 'description': 'Bioengineering'},
 {'code': 'BINF', 'description': 'Bioinformatics'},

In [4]:
# Define the URL and term code
url = "https://nubanner.neu.edu/StudentRegistrationSsb/ssb/term/search"
term_code = "202530"

# Headers for the request
headers = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
}

# Data for the request body
data = {
    "term": term_code,
    "studyPath": "",
    "studyPathText": "",
    "startDatepicker": "",
    "endDatepicker": ""
}

# Make the POST request
post_response = requests.post(url, headers=headers, data=data)

# Check the response
if post_response.status_code != 200:
    print(f"Failed to declare term. Status code: {post_response.status_code}")
    print(post_response.text)

# Get cookie data
print("Cookies:", post_response.cookies)
cookie_jar = post_response.cookies
cookies_dict = {cookie.name: cookie.value for cookie in cookie_jar}
cookies_dict


Cookies: <RequestsCookieJar[<Cookie JSESSIONID=CD77EE06E56AA37500DA36E9D43DA6D2 for nubanner.neu.edu/StudentRegistrationSsb>, <Cookie nubanner-cookie=2334073243.36895.0000 for nubanner.neu.edu/>]>


{'JSESSIONID': 'CD77EE06E56AA37500DA36E9D43DA6D2',
 'nubanner-cookie': '2334073243.36895.0000'}

In [5]:
# Reset search
# reset_url = "https://nubanner.neu.edu/StudentRegistrationSsb/ssb/classSearch/resetDataForm"

# reset_post_response = requests.post(reset_url, cookies_dict)
# reset_post_response.json()

In [6]:
# Define the base URL for course search
base_url = "https://nubanner.neu.edu/StudentRegistrationSsb/ssb/searchResults/searchResults"

# Define the search parameters
params = {
    "txt_subject": "CS",
    "txt_courseNumber": "5800",
    "txt_term": "202530",  # Use the term code previously declared
    "pageOffset": 0,
    "pageMaxSize": 10,
    "sortColumn": "subjectDescription",
    "sortDirection": "asc"
}

# Session headers and cookies (if any cookies were set from the previous term declaration)
headers = {
    "Content-Type": "application/json"
}

# Make the GET request to retrieve course data
class_response = requests.get(base_url, headers=headers, params=params, cookies=cookies_dict)

# Check the response
if class_response.status_code != 200:
    print(f"Failed to retrieve course data. Status code: {class_response.status_code}")
    print(class_response.json())

len(class_response.json()["data"])# [0]["meetingsFaculty"]#[0]["meetingTime"]

10

In [7]:
meeting_infos_list = []

cr_json_data = class_response.json()["data"]

for cur_section in cr_json_data:
    cur_section_meetings = []
    all_meetings_info = cur_section["meetingsFaculty"]

    for cur_meeting in all_meetings_info:
        cur_meeting_time_info = cur_meeting["meetingTime"]
        if cur_meeting_time_info["meetingType"] == "CLAS":
            cur_meeting_dict = {"buildingDescription": cur_meeting_time_info["buildingDescription"],
                                "beginTime": cur_meeting_time_info["beginTime"], 
                                "endTime": cur_meeting_time_info["endTime"],
                                "monday": cur_meeting_time_info["monday"],
                                "tuesday": cur_meeting_time_info["tuesday"],
                                "wednesday": cur_meeting_time_info["wednesday"],
                                "thursday": cur_meeting_time_info["thursday"],
                                "friday": cur_meeting_time_info["friday"]}
        
            cur_section_meetings.append(cur_meeting_dict)

    meeting_infos_list.append(cur_section_meetings)

meeting_infos_list

    
    


[[{'buildingDescription': 'Mugar Life Science Building',
   'beginTime': '1030',
   'endTime': '1135',
   'monday': True,
   'tuesday': False,
   'wednesday': True,
   'thursday': True,
   'friday': False}],
 [{'buildingDescription': 'Churchill Hall',
   'beginTime': '1145',
   'endTime': '1325',
   'monday': True,
   'tuesday': False,
   'wednesday': False,
   'thursday': True,
   'friday': False}],
 [{'buildingDescription': 'Churchill Hall',
   'beginTime': '1450',
   'endTime': '1630',
   'monday': True,
   'tuesday': False,
   'wednesday': True,
   'thursday': False,
   'friday': False}],
 [{'buildingDescription': None,
   'beginTime': None,
   'endTime': None,
   'monday': False,
   'tuesday': False,
   'wednesday': False,
   'thursday': False,
   'friday': False}],
 [{'buildingDescription': 'Roux Institute 100 Fore St.',
   'beginTime': '1300',
   'endTime': '1620',
   'monday': False,
   'tuesday': True,
   'wednesday': False,
   'thursday': False,
   'friday': False}],
 [{'buil

In [8]:
class_df = pd.DataFrame(class_response.json()["data"])
class_df

,id,term,termDesc,courseReferenceNumber,partOfTerm,courseNumber,subject,subjectDescription,sequenceNumber,campusDescription,...,openSection,linkIdentifier,isSectionLinked,subjectCourse,faculty,meetingsFaculty,reservedSeatSummary,sectionAttributes,instructionalMethod,instructionalMethodDescription
0,441515,202530,Spring 2025 Semester,30537,2,5800,CS,Computer Science,01,Boston,...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional
1,446888,202530,Spring 2025 Semester,35910,2,5800,CS,Computer Science,02,Boston,...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional
2,445256,202530,Spring 2025 Semester,34278,2,5800,CS,Computer Science,03,Boston,...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional
3,445268,202530,Spring 2025 Semester,34290,2,5800,CS,Computer Science,04,Online,...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,OL,Online
4,463976,202530,Spring 2025 Semester,40570,2,5800,CS,Computer Science,05,"Portland, Maine",...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional
5,463977,202530,Spring 2025 Semester,40571,2,5800,CS,Computer Science,06,"Portland, Maine",...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional
6,463978,202530,Spring 2025 Semester,40572,2,5800,CS,Computer Science,07,"Portland, Maine",...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional
7,464335,202530,Spring 2025 Semester,40643,2,5800,CS,Computer Science,09,"Seattle, WA",...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional
8,464336,202530,Spring 2025 Semester,40644,2,5800,CS,Computer Science,10,"Seattle, WA",...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional
9,464337,202530,Spring 2025 Semester,40645,2,5800,CS,Computer Science,11,"Seattle, WA",...,True,None,False,CS5800,[],"[{'category': '01', 'class': 'net.hedtech.bann...",None,[{'class': 'net.hedtech.banner.student.schedul...,TR,Traditional


In [9]:
class_df.drop(["term", "termDesc", "partOfTerm", "courseNumber", "subject", "subjectDescription", "linkIdentifier", "isSectionLinked", 
               "instructionalMethodDescription", "meetingsFaculty", "reservedSeatSummary", "sectionAttributes", "crossListCapacity", "crossListCount",
               "crossListAvailable", "enrollment", "creditHourHigh", "creditHourIndicator", "crossList", "creditHours", "waitCapacity",
               "waitCount", "id", "openSection", "scheduleTypeDescription"], 
               axis=1, inplace=True)

class_df["meetingTimes"] = meeting_infos_list
print(class_df.iloc[3].meetingTimes)
class_df

[{'buildingDescription': None, 'beginTime': None, 'endTime': None, 'monday': False, 'tuesday': False, 'wednesday': False, 'thursday': False, 'friday': False}]


,courseReferenceNumber,sequenceNumber,campusDescription,courseTitle,maximumEnrollment,seatsAvailable,waitAvailable,creditHourLow,subjectCourse,faculty,instructionalMethod,meetingTimes
0,30537,01,Boston,Algorithms,150,4,80,4,CS5800,[],TR,[{'buildingDescription': 'Mugar Life Science B...
1,35910,02,Boston,Algorithms,113,2,80,4,CS5800,[],TR,"[{'buildingDescription': 'Churchill Hall', 'be..."
2,34278,03,Boston,Algorithms,79,3,50,4,CS5800,[],TR,"[{'buildingDescription': 'Churchill Hall', 'be..."
3,34290,04,Online,Algorithms,79,7,50,4,CS5800,[],OL,"[{'buildingDescription': None, 'beginTime': No..."
4,40570,05,"Portland, Maine",Algorithms,32,14,15,4,CS5800,[],TR,[{'buildingDescription': 'Roux Institute 100 F...
5,40571,06,"Portland, Maine",Algorithms,32,17,15,4,CS5800,[],TR,[{'buildingDescription': 'Roux Institute 100 F...
6,40572,07,"Portland, Maine",Algorithms,8,4,5,4,CS5800,[],TR,[{'buildingDescription': 'Roux Institute 100 F...
7,40643,09,"Seattle, WA",Algorithms,45,5,20,4,CS5800,[],TR,"[{'buildingDescription': '401 Terry Ave', 'beg..."
8,40644,10,"Seattle, WA",Algorithms,45,31,20,4,CS5800,[],TR,"[{'buildingDescription': '225 Terry Ave', 'beg..."
9,40645,11,"Seattle, WA",Algorithms,45,5,20,4,CS5800,[],TR,"[{'buildingDescription': '225 Terry Ave', 'beg..."


Note: you may need to restart the kernel to use updated packages.
